# Wiener's Attack

### Group 7: Austin Beard, Junior Saintcius, John Owens

In [5]:
# Press ctrl+enter to run code
x = var('x')
solve(x^2 + 3*x + 2, x)

[x == -2, x == -1]

## [Link to Wiener's Attack](https://en.wikipedia.org/wiki/Wiener%27s_attack)

In [7]:
def Wiener(N, e):
    """
    Input: integers N and e
    Output: integers p, q, and d
    """
    
    #Do the attack here
    ...
    
    return p, q, d
    
    
# Group 7's (N, e) pair
# |N| = 4096 bits
N =  947915625495728052916879147697360002848381185192346505887337318484424613804479379229500825154943023456531602566274207306369591959850608246425659384399592991634207719975226030904151882757589795301726622205405989011351842286767545223253169802897983637243167859583565776182244623125858428782210670308167292094521748561446916209557796073456184675807739356933731724960127222046486932110371797638731014250317825447454900437565086817233575744240640382684548268127956137173595899977563370261355506081974467333993545072663993466761303977372442140485495352997067642695566437175979143720808011678287958025730556920566713423095016601310050215390374360676665989625828596980532017231388237244123248528625713956773078115159965411732942413787894237594334962280338475577955633910836559629956564026617798854751095370708381339497715897167336403274224664697600147738325680779222350682292639751395534249345846969763993219058901236669322261406103455020265135135221755871135240191771715762695473152226592913073165991509715785601507547540424028508272006742607644751844712565734384457757831906046716752425927716813761510281226714973672364905890290774159582494385327020386478310127596750245584058550668842273035173930485514220124363571210008854397877759878109
# |e| = 4096 bits
e =  704547874817628536678711224260347170080780157225302414633058087631273478699354942316191545183552970491256712479930884728272203750707194999236141913284914475190760565306955811426771755057311324861065120995469067811477070684340373917672377871037102399678405113518962175705845425490703244686038865941492819081360216141451957694835143605736374176464955479837962508015825872034637667723449908810156455398586426946187983417091803239944119556424937448702560147449296000358404512385942224137056969472773149280435494014441168305623041946847983160401839626890022631063357714302185156647516439105432586919947268965890950159670021155797827170793898880619668912414078180761247776607171716089112584888939756383508383552798449319245738264857608637364581842457448498549984823842222174304406071636079197079103741311287709390835892500282167064692991655929368336712543591994988841130885756415329962100690973384132459114994258583928548266879551395010165988053756662119802587150754631529097435381252816428859268931378932500218048710530873387464697995565066027920879230108859236321171012617250021403409639363576159429016313582199500594524217662031829176343714208316657171662175877320736019254117219869310765030193442816632528910296054592307278119395320563

#p, q, d = Wiener(N, e)

In [5]:
#Test continued fraction (example from Wikipedia) - works
N = 90581
e = 17993
print(continued_fraction(e/N))

[0; 5, 29, 4, 1, 3, 2, 4, 3]


In [8]:
#Testing on multiple (N, e) pairs

def gen_prime(bits):
    #If generating N = 2m bits, we generate primes of m bits each
    
    p = ...
    q = ...
    
    return p * q

N = gen_prime(bits)

p, q, d = Wiener(N, e)


NameError: name 'bits' is not defined